<a href="https://colab.research.google.com/github/garrisonreid/MassShooterMap/blob/main/MassShooterMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
# Packages for the dataframe import and mapping
import pandas as pd
import folium

In [179]:
# Connects the spreadsheet
df = pd.read_excel("https://docs.google.com/spreadsheets/d/e/2PACX-1vS688m__6C7oPWQtiiI4WB_ojGP6cmsUjskIKt6CYuWjHJxoXNT_7fIOWcvzp8rpy4cJEN0N0zH_EKY/pub?output=xlsx", sheet_name="Full Database")

# 2nd row was actually the column headers
df.columns = df.iloc[0]

# Skip the row that is now the headers
df = df[1:]

# Previews the dataframe
df.head()

,Case #,Shooter Last Name,Shooter First Name,Full Date,Day of Week,Day,Month,Year,Shooting Location Address,City,State,Region,Urban/Suburban/Rural,Location,Insider / Outsider,Bifurcated,"If Bifurcated, Other Location",Armed Person on Scene,Specify Armed Person,Number Killed,Number Injured,Kidnapping or Hostage Situation,Age,Gender,Race,Immigrant,Sexual Orientation,Religion,Education,School Performance,School Performance Specified,Birth Order,Number of Siblings,Older Siblings,Younger Siblings,Relationship Status,Children,Employment Status,Employment Type,Military Service,...,Known Prejudice 3,Motive: Racism/Xenophobia,Motive: Religious Hate,Motive: Misogyny,Motive: Homophobia\n,Motive: Employment Issue,Motive: Economic Issue,Motive: Legal Issue,Motive: Relationship Issue,Motive: Interpersonal Conflict,Motive: Fame-Seeking,Motive: Other,Motive: Unknown,Domestic Spillage,Role of Psychosis in the Shooting,Social Media Use,Leakage,Leakage 1 How,Leakage 1 Who,Leakage 1 Specific/Nonspecific,Leakage 2 How,Leakage 2 Who,Leakage 2 Specific/Nonspecific,Interest in Past Mass Violence,Relationship with Other Shooting(s),Specify Relationship to Other Shooting(s),Legacy Token,Pop Culture Connection,Specify Pop Culture Connection,Planning,Performance,Interest in Firearms,Firearm Proficiency,Total Firearms Brought to the Scene,Other Weapons or Gear,Specify Other Weapons or Gear,On-Scene Outcome,Attempt to Flee,Insanity Defense,Criminal Sentence
1,1,Whitman,Charles,1966-08-01 00:00:00,Monday,1,8,1966,"110 Inner Campus Drive, Austin, TX 78705",Austin,43,0,0,1,1,1,7,1,2,15,31,0,25,0,0,0,0,1,2,0,1.9 GPA,1,2,0,2,2,0,0,NaN,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,1,0,0,1,NaN,NaN,NaN,0,1,Robert Smith killed 5 people a few months afte...,1,0,NaN,1,0,1,3,7,1,"hatchet, hammer, knives, wrench, ropes, water,...",1,0,2,0
2,2,Smith,Robert,1966-11-12 00:00:00,Saturday,12,11,1966,"Rose-Mar College of Beauty in Mesa, AZ",Mesa,3,3,0,4,0,0,NaN,0,0,5,2,0,18,0,0,0,0,0,0,2,"good grades, on student council",1,1,0,1,0,0,0,NaN,0,...,NaN,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Happened a few months after the Charles Whitma...,0,0,NaN,1,1,0,1,1,1,"knife, nylon cord",2,0,1,1
3,3,Held,Leo,1967-10-23 00:00:00,Monday,23,10,1967,"599 South Highland Street Lockhaven, PA17745",Lock Haven,38,2,2,6,1,1,7,0,0,6,6,0,39,0,0,0,0,1,2,NaN,NaN,NaN,2,NaN,NaN,2,1,1,2,1,...,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,1,3,2,1,holster,1,0,2,0
4,4,Pearson,Eric,1968-03-16 00:00:00,Saturday,16,3,1968,"11703 Lake Rd, Ironwood, MI 49938",Ironwood,22,0,2,5,0,1,7,0,0,7,2,0,56,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,...,NaN,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,0,NaN,1,0,NaN,2,0,0,3
5,5,Lambright,Donald,1969-04-05 00:00:00,Saturday,5,4,1969,"Pennsylvania Turnpike near Harrisburg, PA",Harrisburg,38,2,0,8,0,0,NaN,0,0,4,17,NaN,31,0,1,0,0,1,2,2,"brilliant, attended several colleges",0,0,0,0,2,1,0,2,1,...,NaN,0,0,0,0,0,0,0,0,0,0,1,0,1,1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,0,3,2,0,NaN,0,0,2,0


In [180]:
# 2nd spreadsheet tab that has the lat/long data
longlat = pd.read_excel("https://docs.google.com/spreadsheets/d/e/2PACX-1vS688m__6C7oPWQtiiI4WB_ojGP6cmsUjskIKt6CYuWjHJxoXNT_7fIOWcvzp8rpy4cJEN0N0zH_EKY/pub?output=xlsx", sheet_name="LongLat")

# Merges the two dataframes based on a common column of data (Case #)
df = longlat.merge(df, left_on = 'Case #', right_on = 'Case #')

# Previews the merged dataframe
df.head()

,Case #,Shooting Location Address_x,lat,lon,0,Shooter Last Name,Shooter First Name,Full Date,Day of Week,Day,Month,Year,Shooting Location Address_y,City,State,Region,Urban/Suburban/Rural,Location,Insider / Outsider,Bifurcated,"If Bifurcated, Other Location",Armed Person on Scene,Specify Armed Person,Number Killed,Number Injured,Kidnapping or Hostage Situation,Age,Gender,Race,Immigrant,Sexual Orientation,Religion,Education,School Performance,School Performance Specified,Birth Order,Number of Siblings,Older Siblings,Younger Siblings,Relationship Status,...,Known Prejudice 3,Motive: Racism/Xenophobia,Motive: Religious Hate,Motive: Misogyny,Motive: Homophobia\n,Motive: Employment Issue,Motive: Economic Issue,Motive: Legal Issue,Motive: Relationship Issue,Motive: Interpersonal Conflict,Motive: Fame-Seeking,Motive: Other,Motive: Unknown,Domestic Spillage,Role of Psychosis in the Shooting,Social Media Use,Leakage,Leakage 1 How,Leakage 1 Who,Leakage 1 Specific/Nonspecific,Leakage 2 How,Leakage 2 Who,Leakage 2 Specific/Nonspecific,Interest in Past Mass Violence,Relationship with Other Shooting(s),Specify Relationship to Other Shooting(s),Legacy Token,Pop Culture Connection,Specify Pop Culture Connection,Planning,Performance,Interest in Firearms,Firearm Proficiency,Total Firearms Brought to the Scene,Other Weapons or Gear,Specify Other Weapons or Gear,On-Scene Outcome,Attempt to Flee,Insanity Defense,Criminal Sentence
0,1,"110 Inner Campus Drive, Austin, TX 78705",30.286095,-97.739297,1.00,Whitman,Charles,1966-08-01 00:00:00,Monday,1,8,1966,"110 Inner Campus Drive, Austin, TX 78705",Austin,43,0,0,1,1,1,7,1,2,15,31,0,25,0,0,0,0,1,2,0,1.9 GPA,1,2,0,2,2,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,1,0,0,1,NaN,NaN,NaN,0,1,Robert Smith killed 5 people a few months afte...,1,0,NaN,1,0,1,3,7,1,"hatchet, hammer, knives, wrench, ropes, water,...",1,0,2,0
1,2,"Rose-Mar College of Beauty in Mesa, AZ",33.459937,-111.781900,0.90,Smith,Robert,1966-11-12 00:00:00,Saturday,12,11,1966,"Rose-Mar College of Beauty in Mesa, AZ",Mesa,3,3,0,4,0,0,NaN,0,0,5,2,0,18,0,0,0,0,0,0,2,"good grades, on student council",1,1,0,1,0,...,NaN,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Happened a few months after the Charles Whitma...,0,0,NaN,1,1,0,1,1,1,"knife, nylon cord",2,0,1,1
2,3,"599 South Highland Street Lockhaven, PA17745",41.126372,-77.452856,0.27,Held,Leo,1967-10-23 00:00:00,Monday,23,10,1967,"599 South Highland Street Lockhaven, PA17745",Lock Haven,38,2,2,6,1,1,7,0,0,6,6,0,39,0,0,0,0,1,2,NaN,NaN,NaN,2,NaN,NaN,2,...,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,1,3,2,1,holster,1,0,2,0
3,4,"11703 Lake Rd, Ironwood, MI 49938",46.522175,-90.156243,1.00,Pearson,Eric,1968-03-16 00:00:00,Saturday,16,3,1968,"11703 Lake Rd, Ironwood, MI 49938",Ironwood,22,0,2,5,0,1,7,0,0,7,2,0,56,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,0,NaN,1,0,NaN,2,0,0,3
4,5,"Pennsylvania Turnpike near Harrisburg, PA",40.076302,-75.537477,0.40,Lambright,Donald,1969-04-05 00:00:00,Saturday,5,4,1969,"Pennsylvania Turnpike near Harrisburg, PA",Harrisburg,38,2,0,8,0,0,NaN,0,0,4,17,NaN,31,0,1,0,0,1,2,2,"brilliant, attended several colleges",0,0,0,0,2,...,NaN,0,0,0,0,0,0,0,0,0,0,1,0,1,1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,0,0,0,3,2,0,NaN,0,0,2,0


In [181]:
# Adds "N/A" to the spaces that were missing data
df['Race'] = df['Race'].fillna("N/A")

# This is a goofy way, but I took the data dictionary values and replaced 0-6 values with the actual ethnicities
di = {0: "White", 1: "Black", 2: "LatinX", 3: "Asian", 4: "Middle Eastern", 5: "Native American", 6: "Other"}
df = df.replace({"Race": di})

# Also adds "N/A" to another column that appears to have blanks
df['Specify Other Weapons or Gear'] = df['Specify Other Weapons or Gear'].fillna("N/A")

In [182]:
# Creates a blank map
m = folium.Map(location=[df['lat'].mean(),df['lon'].mean()], tiles=None, zoom_start=5)
# Adds the OpenStreetMap tiles
folium.TileLayer (tiles = 'openstreetmap', show=True, control=False, min_zoom=3).add_to(m)

# A dozen colors from ColorBrewer that work in maps
fg_colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a']

# Flexible value to adjust the size of the circles
multiplier = 2500

# Outer loop that will go through the unique options in the Race column
for index, i in enumerate(sorted(df.Race.unique())):
  # Feature Groups are the toggles
  fg = folium.FeatureGroup(name=str(i), show=True)
  # Filters the entire dataset to only where the race is the value of that iteration of the loop
  temp_df = df[df.Race == i]

  # Inner loop: Goes through each point in the filtered dataframe
  for i in range(len(temp_df)):
    # Creates a marker...
    folium.Circle(
        # at that location...
        location=[temp_df.iloc[i]['lat'], temp_df.iloc[i]['lon']],
        # with the relevant info for that marker...
        tooltip="<strong>Date: </strong>" + str(temp_df.iloc[i]['Day']) + "/" + str(temp_df.iloc[i]['Month']) + "/" + str(temp_df.iloc[i]['Year']) + 
                "<br><strong>Address: </strong>"+str(temp_df.iloc[i]['Shooting Location Address_x']) + 
                "<br><strong>Number Killed: </strong>" + str(temp_df.iloc[i]['Number Killed']) + 
                "<br><strong>Race: </strong>" + str(temp_df.iloc[i]['Race']) + 
                "<br><strong>Age: </strong>" + str(temp_df.iloc[i]['Age']) + 
                "<br><strong>Specify Other Weapons or Gear: </strong>" + str(temp_df.iloc[i]['Specify Other Weapons or Gear']),
        # in the size of the datapoint's number killed * the multiplier
        radius=float(temp_df.iloc[i]['Number Killed'])*multiplier,
        fill=True,
        # in the color at the position of the times through the loop
        fill_color=fg_colors[index],
        fill_opacity = 0.75,
        stroke = False,
        control = True
    # Adds those points to the feature group
    ).add_to(fg)
  # Adds that feature group to the map
  fg.add_to(m)
  # NEXT TIME THROUGH THE LOOP, fresh/empty featuregroup, adds all the points

# Adds toggle controls to the map
folium.LayerControl(collapsed=False).add_to(m)
# Shows the map
m

In [183]:
# When looking for age I grouped ages by deca. 
df['AgeGroup'] = df.Age // 10 # Divides by 10, rounds down 34 --> 3

# Same goofy process of matching values to their meeting
di = {0: "N/A", 1: "10s", 2: "20s", 3: "30s", 4: "40s", 5: "50s", 6: "60s", 7: "70s"}
df = df.replace({"AgeGroup": di})

# Adds a N/A for nulls, shows all the values as strings/text vs. numerical values
df['AgeGroup'] = df['AgeGroup'].fillna("N/A").astype(str)

# Shows the unique values in the column to confirm there's no nulls
df.AgeGroup.unique()

array(['20s', '10s', '30s', '50s', '40s', '70s', '60s', 'N/A'],
      dtype=object)

In [184]:
m2 = folium.Map(location=[df['lat'].mean(),df['lon'].mean()], tiles=None, zoom_start=5)
folium.TileLayer (tiles = 'openstreetmap', show=True, control=False, min_zoom=3).add_to(m2)

fg_colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a']

multiplier = 2500

for index, i in enumerate(sorted(df.AgeGroup.unique())):
  fg = folium.FeatureGroup(name= str(i), show=True)
  temp_df = df[df.AgeGroup == i]
  for i in range(len(temp_df)):
    folium.Circle(
        location=[temp_df.iloc[i]['lat'], temp_df.iloc[i]['lon']],
        tooltip="<strong>Date: </strong>" + str(temp_df.iloc[i]['Day']) + "/" + str(temp_df.iloc[i]['Month']) + "/" + str(temp_df.iloc[i]['Year']) + 
                "<br><strong>Address: </strong>"+str(temp_df.iloc[i]['Shooting Location Address_x']) + 
                "<br><strong>Number Killed: </strong>" + str(temp_df.iloc[i]['Number Killed']) + 
                "<br><strong>Race: </strong>" + str(temp_df.iloc[i]['Race']) + 
                "<br><strong>Age: </strong>" + str(temp_df.iloc[i]['Age']) + 
                "<br><strong>Specify Other Weapons or Gear: </strong>" + str(temp_df.iloc[i]['Specify Other Weapons or Gear']),        radius=float(temp_df.iloc[i]['Number Killed'])*multiplier,
        color=fg_colors[index],
        stroke = False,
        fill=True,
        fill_color=fg_colors[index],
        fill_opacity = 0.75,
        control = True
    ).add_to(fg)
  fg.add_to(m2)

folium.LayerControl(collapsed=False).add_to(m2)
m2

In [185]:
# Saving the maps from the Colab to standalone .html files
from google.colab import files
m.save("MapByRace.html")
m2.save("MapByAge.html")

# Download the files to your local computer
files.download("MapByRace.html")
files.download("MapByAge.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>